In [279]:
%run my_functions.py


[nltk_data] Downloading package stopwords to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Annelie Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [280]:
# loading preprocessed dataset
with open("../dataframes/raw_data_Books_00_clean_and_cut.pkl", "rb") as f:
    df = pickle.load(f)
df = df[:20000]

In [281]:
texts =[]
for doc in df.cut_and_clean:
        words = tokenizer.tokenize(doc)
        texts.append(words)

dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=20, no_above=0.8)

## 1. Transformers (Bag-of-Words)

### 1.1 Count Vectorizer

In [282]:
# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.8, min_df=20, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf_fit = vectorizer.fit_transform(df['cut_and_clean'])
pickle.dump(tf_fit, open('../models/bow_tf.pkl', 'wb')) #save fitted model
tf = tf_fit.toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



### 1.2 TD-IDF

In [283]:
tfidf_vec = TfidfVectorizer( 
                        max_df=0.8,
                        min_df=20,
                        token_pattern='\w+|\$[\d\.]+|\S+'
                       )
tfidf_fit = tfidf_vec.fit_transform(df['cut_and_clean'])
pickle.dump(tfidf_fit, open('../models/bow_tfidf.pkl', 'wb')) #save fitted model
tfidf = tfidf_fit.toarray()

tfidf_feature_names = tfidf_vec.get_feature_names()

# LSA (SVD)

In [313]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=10, algorithm='arpack')


### LSA TF

In [314]:
lsa_tf = lsa.fit(tf.astype('float64'))
pickle.dump(lsa_tf, open('../models/basemodel_lsa_tf.pkl', 'wb')) #save fitted model



In [315]:
# Evaluation Metrics for whole dataset
cv_lsa_tf, c_umass_lsa_tf, c_npmi_lsa_tf, c_uci_lsa_tf, sil_score_lsa_tf, ch_index_lsa_tf, db_index_lsa_tf = evaluation(lsa_tf, tf_fit, tf, tf_feature_names, 20, df["cut_and_clean"], dictionary)


Coherence Score CV:  0.5012502358053665

Coherence Score UMass:  -2.9575934683886045

Coherence Score NPMI:  -0.022251702366717176

Coherence Score UCI:  -1.482800667471404

Silhouette Score:  -0.18189454640280814

Calinski-Harabasz Score:  18.295407805927802

Davies-Bouldin Score:  7.920377990333579


In [316]:
display_topics(lsa_tf, tf_feature_names, 10)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,life,0.2,jesus,0.4,fine,0.4,heart,0.7,god,0.6,practice,0.3,life,0.5,character,0.6,exercise,0.5,exercise,0.3
1,author,0.2,memory,0.4,social,0.2,red,0.6,life,0.3,teacher,0.3,work,0.2,series,0.3,chapter,0.4,work,0.3
2,god,0.2,word,0.3,violence,0.2,fantasy,0.1,universe,0.3,science,0.2,japanese,0.2,teacher,0.1,page,0.2,year,0.2
3,way,0.2,god,0.3,political,0.2,way,0.1,science,0.2,chapter,0.2,thing,0.1,end,0.1,body,0.2,series,0.2
4,year,0.2,man,0.2,religious,0.2,fine,0.1,law,0.1,education,0.2,self,0.1,science,0.1,character,0.2,thing,0.2
5,work,0.2,heal,0.1,government,0.2,plot,0.1,evidence,0.1,work,0.2,war,0.1,novel,0.1,author,0.1,body,0.1
6,world,0.1,fresh,0.1,religion,0.1,world,0.1,sin,0.1,exercise,0.2,live,0.1,education,0.1,god,0.1,way,0.1
7,page,0.1,imagination,0.1,world,0.1,overall,0.0,christ,0.1,school,0.2,social,0.1,practice,0.1,strength,0.1,fine,0.1
8,new,0.1,john,0.1,test,0.1,previous,0.0,scientific,0.1,research,0.1,character,0.1,plot,0.1,example,0.1,help,0.1
9,day,0.1,false,0.1,nation,0.1,help,0.0,system,0.1,project,0.1,worker,0.1,reader,0.1,work,0.1,character,0.1


In [317]:
data = display_topics(lsa_tf, tf_feature_names, 10)
fig = make_subplots(rows=5, cols=2, subplot_titles= ["Topic 1", "Topic 6", "Topic 2","Topic 7", "Topic 3","Topic 8","Topic 4","Topic 9", "Topic 5", "Topic 10"])
colors = ["Blue", "Pink", "SeaGreen", "Orange", "Grey", "Purple", "Red", "LightBlue", "LightGreen", "DarkRed"]
for col in range(0, int(len(data.columns)),2):
    if col == 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,1].astype("float64"),
        y=data.iloc[:,0],
        marker_color=colors[0], orientation = "h"), col= 1, row = 1)
    if col < 10 and col > 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"), col= 1, row = int(col/2)+1)
    if col >= 10:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"),col= 2, row = int(col/2)-4)

fig.update_layout( height=1400, width=1200, title_text="LSA TF - Topic Keywords ranked by Weight", showlegend=False, xaxis_title_text='Keyword',yaxis_title_text='Weight'
    )
fig.update_yaxes(title='Keyword', type = "category" , autorange = "reversed")
fig.update_xaxes( title = 'Weight', range = (0,1) )
fig.show()

### LSA TF-IDF

In [289]:
lsa_tfidf = lsa.fit(tfidf)
pickle.dump(lsa_tfidf, open('../models/basemodel_lsa_tfidf.pkl', 'wb')) #save fitted model

In [290]:
# Evaluation Metrics for whole dataset
cv_lsa_tfidf, c_umass_lsa_tfidf, c_npmi_lsa_tfidf, c_uci_lsa_tfidf, sil_score_lsa_tfidf, ch_index_lsa_tfidf, db_index_lsa_tfidf = evaluation(lsa_tfidf, tfidf_fit, tfidf, tfidf_feature_names, 20, df["cut_and_clean"], dictionary)


Coherence Score CV:  0.40831600006554086

Coherence Score UMass:  -3.5451028978230283

Coherence Score NPMI:  0.02149260958252702

Coherence Score UCI:  -0.30060269884111207

Silhouette Score:  -0.05878108067727086

Calinski-Harabasz Score:  118.42376673525558

Davies-Bouldin Score:  4.6087010943110505


In [291]:
display_topics(lsa_tfidf, tfidf_feature_names, 10)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,thank,1.0,gift,0.3,condition,0.9,gift,0.9,series,0.8,wonderful,0.8,nice,0.8,helpful,0.9,enjoy,0.7,old,0.4
1,condition,0.2,series,0.3,perfect,0.2,perfect,0.1,gift,0.2,condition,0.1,wonderful,0.4,series,0.2,helpful,0.3,year,0.4
2,perfect,0.1,wonderful,0.2,arrive,0.1,christmas,0.0,enjoy,0.1,gift,0.0,series,0.2,gift,0.1,old,0.2,perfect,0.4
3,gift,0.0,year,0.2,describe,0.0,friend,0.0,condition,0.1,author,0.0,gift,0.0,easy,0.1,year,0.2,daughter,0.2
4,wonderful,0.0,condition,0.2,price,0.0,daughter,0.0,character,0.0,message,0.0,condition,0.0,informative,0.0,son,0.1,helpful,0.1
5,nice,0.0,old,0.2,receive,0.0,purchase,0.0,favorite,0.0,man,0.0,picture,0.0,nice,0.0,nice,0.1,son,0.1
6,arrive,0.0,enjoy,0.2,come,0.0,condition,0.0,wait,0.0,miss,0.0,quality,0.0,information,0.0,wonderful,0.1,nice,0.1
7,product,0.0,easy,0.2,brand,0.0,arrive,0.0,son,0.0,world,0.0,addition,0.0,guide,0.0,kid,0.0,grandson,0.1
8,price,0.0,nice,0.1,pleased,0.0,birthday,0.0,end,0.0,writer,0.0,service,0.0,understand,0.0,daughter,0.0,favorite,0.1
9,service,0.0,lot,0.1,item,0.0,husband,0.0,thank,0.0,writing,0.0,calendar,0.0,useful,0.0,condition,0.0,kid,0.1


In [292]:
data = display_topics(lsa_tfidf, tfidf_feature_names, 10)
fig = make_subplots(rows=5, cols=2, subplot_titles= ["Topic 1", "Topic 6", "Topic 2","Topic 7", "Topic 3","Topic 8","Topic 4","Topic 9", "Topic 5", "Topic 10"])
colors = ["Blue", "Pink", "SeaGreen", "Orange", "Grey", "Purple", "Red", "LightBlue", "LightGreen", "DarkRed"]
for col in range(0, int(len(data.columns)),2):
    if col == 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,1].astype("float64"),#.sort_values(ascending=True)
        y=data.iloc[:,0], #.sort_values( by= data.columns[col+1], ascending = True)
        marker_color=colors[0], orientation = "h"), col= 1, row = 1)
    if col < 10 and col > 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"), col= 1, row = int(col/2)+1)
    if col >= 10:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"),col= 2, row = int(col/2)-4)

fig.update_layout( height=1400, width=1200, title_text="LSA TF-IDF - Topic Keywords ranked by Weight", showlegend=False, xaxis_title_text='Keyword',yaxis_title_text='Weight'
    )
fig.update_yaxes(title='Keyword', type = "category" , autorange = "reversed")
fig.update_xaxes( title = 'Weight', range = (0,1.5) )
fig.show()








# LDA

In [293]:
alpha = 50/10 # 50 / nr of topics
beta = 200/len(dictionary) # 200 / len of vocab

In [294]:
from sklearn.decomposition import LatentDirichletAllocation


lda = LatentDirichletAllocation(n_components=10,random_state=0, learning_method="online", n_jobs=-1, doc_topic_prior=alpha, topic_word_prior=beta)

### LDA TF

In [295]:
lda_tf = lda.fit(tf) #whole dataset
pickle.dump(lda_tf, open('../models/basemodel_lda_tf.pkl', 'wb')) #save fitted model


In [296]:
# Evaluation Metrics for whole dataset
cv_lda_tf, c_umass_lda_tf, c_npmi_lda_tf, c_uci_lda_tf, sil_score_lda_tf, ch_index_lda_tf, db_index_lda_tf = evaluation(lda_tf, tf_fit, tf, tf_feature_names, 20, df["cut_and_clean"], dictionary)


Coherence Score CV:  0.5062649981975438

Coherence Score UMass:  -2.787612311806328

Coherence Score NPMI:  0.0233255101980619

Coherence Score UCI:  0.10160875651382122

Silhouette Score:  -0.15812607799438116

Calinski-Harabasz Score:  21.635619568508357

Davies-Bouldin Score:  13.203559819123422


In [297]:
display_topics(lda_tf, tf_feature_names, 10)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,easy,475.3,character,974.1,heart,380.3,helpful,363.4,day,445.8,child,637.2,enjoy,294.9,recipe,655.2,god,480.7,work,505.8
1,year,402.5,series,674.9,way,338.2,color,358.8,author,365.1,year,476.8,gift,220.1,cook,399.3,war,399.6,student,277.5
2,information,345.4,end,418.3,world,208.0,useful,308.3,chapter,357.4,old,408.3,class,203.8,new,299.5,world,398.0,text,274.8
3,new,277.1,novel,336.4,life,207.3,art,307.4,thing,261.9,life,276.7,old,194.8,order,299.0,human,284.9,use,269.5
4,wonderful,224.4,little,333.9,journey,186.8,page,258.1,life,232.8,son,274.8,wife,189.7,easy,217.3,american,283.3,practice,264.1
5,old,221.0,enjoy,285.7,series,185.7,lot,257.4,feel,225.2,family,273.8,fun,176.3,bible,214.7,life,264.8,change,229.2
6,nice,212.9,page,244.4,mystery,172.1,money,257.3,page,219.0,friend,236.6,daughter,175.0,look,207.0,man,231.1,example,215.5
7,recommend,211.8,lot,224.9,english,171.9,print,252.7,way,206.8,parent,220.5,work,172.7,wonderful,198.9,state,227.2,chapter,212.1
8,perfect,196.2,author,184.2,author,170.7,product,240.7,short,187.5,girl,220.4,new,168.0,husband,176.7,view,222.3,research,201.6
9,step,193.5,plot,176.2,reader,152.6,purchase,239.6,plan,183.1,age,206.1,worth,155.3,thank,173.1,christian,216.5,level,171.2


In [298]:
data = display_topics(lda_tf, tf_feature_names, 10)
fig = make_subplots(rows=5, cols=2, subplot_titles= ["Topic 1", "Topic 6", "Topic 2","Topic 7", "Topic 3","Topic 8","Topic 4","Topic 9", "Topic 5", "Topic 10"])
colors = ["Blue", "Pink", "SeaGreen", "Orange", "Grey", "Purple", "Red", "LightBlue", "LightGreen", "DarkRed"]
for col in range(0, int(len(data.columns)),2):
    if col == 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,1].astype("float64"),#.sort_values(ascending=True)
        y=data.iloc[:,0], #.sort_values( by= data.columns[col+1], ascending = True)
        marker_color=colors[0], orientation = "h"), col= 1, row = 1)
    if col < 10 and col > 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"), col= 1, row = int(col/2)+1)
    if col >= 10:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"),col= 2, row = int(col/2)-4)

fig.update_layout( height=1400, width=1200, title_text="LDA TF - Topic Keywords ranked by Weight", showlegend=False, xaxis_title_text='Keyword',yaxis_title_text='Weight'
    )
fig.update_yaxes(title='Keyword', type = "category" , autorange = "reversed")
fig.update_xaxes( title = 'Weight', range = (0,1000) )
fig.show()

### LDA TF-IDF

In [299]:
lda_tfidf = lda.fit(tfidf)
pickle.dump(lda_tfidf, open('../models/basemodel_lda_tfidf.pkl', 'wb')) #save fitted model

In [300]:
# Evaluation Metrics for whole dataset
cv_lda_tfidf, c_umass_lda_tfidf, c_npmi_lda_tfidf, c_uci_lda_tfidf, sil_score_lda_tfidf, ch_index_lda_tfidf, db_index_lda_tfidf = evaluation(lda_tfidf, tfidf_fit, tfidf, tfidf_feature_names, 20, df["cut_and_clean"], dictionary)


Coherence Score CV:  0.3853283133699152

Coherence Score UMass:  -3.3565646896529797

Coherence Score NPMI:  -0.014480702492772713

Coherence Score UCI:  -0.3977433778296897

Silhouette Score:  -0.046083961251919485

Calinski-Harabasz Score:  16.23536465959202

Davies-Bouldin Score:  18.270007042187324


In [301]:
display_topics(lda_tfidf, tfidf_feature_names, 10)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,year,44.6,gift,59.7,thank,75.0,author,47.5,wonderful,42.8,thank,36.4,enjoy,40.0,thank,52.1,free,34.2,thank,53.3
1,easy,40.5,enjoy,45.6,year,39.7,helpful,44.4,day,29.8,year,35.4,gift,39.6,wonderful,38.4,condition,34.0,series,40.2
2,perfect,39.9,thank,36.3,series,38.8,happy,36.2,kid,29.7,child,31.3,condition,37.9,recipe,37.1,sure,30.8,condition,37.3
3,new,39.0,nice,36.2,nice,34.9,idea,36.2,birthday,28.0,condition,31.0,series,37.5,gift,35.4,life,29.5,nice,37.0
4,thank,36.3,series,34.3,resource,32.8,thank,35.1,author,27.0,lot,29.3,deliver,34.6,husband,34.0,favorite,29.4,work,36.6
5,wonderful,30.6,lot,32.7,journey,31.6,easy,34.1,year,25.3,series,28.4,work,32.6,new,33.0,thank,28.5,price,31.8
6,gift,30.4,condition,30.4,lot,31.4,lot,33.4,recommend,25.1,pleased,28.4,thank,30.6,author,32.1,information,28.5,wonderful,31.0
7,information,30.3,page,29.7,reference,31.4,important,31.0,finish,24.9,wonderful,28.3,old,30.3,year,31.8,product,27.6,gift,30.9
8,helpful,28.9,work,28.9,gift,31.3,need,30.9,thank,24.9,perfect,28.1,character,30.1,fun,31.4,series,27.5,stuff,30.6
9,practical,28.1,character,28.3,easy,31.0,fun,28.3,gift,24.8,life,27.2,new,29.2,information,29.6,dog,27.0,help,29.3


In [302]:
data = display_topics(lda_tfidf, tfidf_feature_names, 10)
fig = make_subplots(rows=5, cols=2, subplot_titles= ["Topic 1", "Topic 6", "Topic 2","Topic 7", "Topic 3","Topic 8","Topic 4","Topic 9", "Topic 5", "Topic 10"])
colors = ["Blue", "Pink", "SeaGreen", "Orange", "Grey", "Purple", "Red", "LightBlue", "LightGreen", "DarkRed"]
for col in range(0, int(len(data.columns)),2):
    if col == 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,1].astype("float64"),#.sort_values(ascending=True)
        y=data.iloc[:,0], #.sort_values( by= data.columns[col+1], ascending = True)
        marker_color=colors[0], orientation = "h"), col= 1, row = 1)
    if col < 10 and col > 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"), col= 1, row = int(col/2)+1)
    if col >= 10:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"),col= 2, row = int(col/2)-4)

fig.update_layout( height=1400, width=1200, title_text="LDA TF-IDF - Topic Keywords ranked by Weight", showlegend=False, xaxis_title_text='Keyword',yaxis_title_text='Weight'
    )
fig.update_yaxes(title='Keyword', type = "category" , autorange = "reversed")
fig.update_xaxes( title = 'Weight', range = (0,100) )
fig.show()

# Non-Negative Matrix Factorization

In [303]:
from sklearn.decomposition import NMF
nmf = MiniBatchNMF(n_components=10, init= 'nndsvd')

### NMF TF

In [304]:
NMF_tf = nmf.fit(tf)
pickle.dump(NMF_tf, open('../models/basemodel_nmf_tf.pkl', 'wb')) #save fitted model

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.



In [305]:
# Evaluation Metrics for whole dataset
cv_nmf_tf, c_umass_nmf_tf, c_npmi_nmf_tf, c_uci_nmf_tf, sil_score_nmf_tf, ch_index_nmf_tf, db_index_nmf_tf = evaluation(NMF_tf, tf_fit, tf, tf_feature_names, 20, df["cut_and_clean"], dictionary)


Coherence Score CV:  0.5821935337743727

Coherence Score UMass:  -2.7899509016571007

Coherence Score NPMI:  0.01601401147259798

Coherence Score UCI:  -0.6607062708477225

Silhouette Score:  -0.020699944647280373

Calinski-Harabasz Score:  36.62382060671058

Davies-Bouldin Score:  8.438336836356731


In [306]:
display_topics(NMF_tf, tf_feature_names, 10)


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,author,3.2,jesus,5.4,fine,4.9,heart,7.2,god,6.7,teacher,3.1,life,9.9,character,7.6,page,10.7,year,5.9
1,way,2.3,memory,4.8,violence,2.8,red,6.4,universe,3.6,practice,3.0,live,1.8,series,4.6,chapter,4.7,work,4.6
2,day,2.2,word,3.7,social,2.6,way,0.8,life,2.3,education,2.5,war,1.6,end,1.9,exercise,1.5,old,3.8
3,year,2.1,god,2.7,political,2.0,fantasy,0.7,science,2.2,science,2.2,work,1.4,novel,1.6,example,1.5,thing,2.6
4,new,1.9,man,2.3,religious,1.9,plot,0.6,law,1.3,school,1.8,change,1.3,plot,1.1,author,1.4,way,2.3
5,world,1.8,heal,1.6,government,1.8,help,0.5,evidence,1.3,research,1.7,self,1.2,scene,0.9,version,1.0,recipe,2.2
6,little,1.8,john,1.6,world,1.8,overall,0.5,sin,1.1,project,1.5,thing,1.1,reader,0.9,section,0.9,cook,1.9
7,life,1.8,fresh,1.5,religion,1.6,previous,0.5,scientific,1.0,classroom,1.4,way,1.0,main,0.7,color,0.9,help,1.7
8,series,1.7,imagination,1.5,test,1.3,world,0.4,system,0.8,work,1.4,japanese,1.0,problem,0.6,print,0.8,exercise,1.3
9,work,1.6,false,1.4,anti,1.3,end,0.4,new,0.8,direct,1.3,new,0.8,lot,0.6,understand,0.8,easy,1.3


In [308]:
data = display_topics(NMF_tf, tf_feature_names, 10)
fig = make_subplots(rows=5, cols=2, subplot_titles= ["Topic 1", "Topic 6", "Topic 2","Topic 7", "Topic 3","Topic 8","Topic 4","Topic 9", "Topic 5", "Topic 10"])
colors = ["Blue", "Pink", "SeaGreen", "Orange", "Grey", "Purple", "Red", "LightBlue", "LightGreen", "DarkRed"]
for col in range(0, int(len(data.columns)),2):
    if col == 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,1].astype("float64"),#.sort_values(ascending=True)
        y=data.iloc[:,0], #.sort_values( by= data.columns[col+1], ascending = True)
        marker_color=colors[0], orientation = "h"), col= 1, row = 1)
    if col < 10 and col > 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"), col= 1, row = int(col/2)+1)
    if col >= 10:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"),col= 2, row = int(col/2)-4)

fig.update_layout( height=1400, width=1200, title_text="NMF TF - Topic Keywords ranked by Weight", showlegend=False, xaxis_title_text='Keyword',yaxis_title_text='Weight'
    )
fig.update_yaxes(title='Keyword', type = "category" , autorange = "reversed")
fig.update_xaxes( title = 'Weight', range = (0,10) )
fig.show()

### NMF TF-IDF

In [309]:
NMF_tfidf = nmf.fit(tfidf)
pickle.dump(NMF_tfidf, open('../models/basemodel_nmf_tfidf.pkl', 'wb')) #save fitted model

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.



In [310]:
# Evaluation Metrics for whole dataset
cv_nmf_tfidf, c_umass_nmf_tfidf, c_npmi_nmf_tfidf, c_uci_nmf_tfidf,  sil_score_nmf_tfidf, ch_index_nmf_tfidf, db_index_nmf_tfidf = evaluation(NMF_tfidf, tfidf_fit, tfidf, tfidf_feature_names, 20, df["cut_and_clean"], dictionary)


Coherence Score CV:  0.40194805641515413

Coherence Score UMass:  -3.745690564789002

Coherence Score NPMI:  0.024130478726421097

Coherence Score UCI:  -0.36982125544336963

Silhouette Score:  -0.08690528493234113

Calinski-Harabasz Score:  119.13018673681427

Davies-Bouldin Score:  4.854282211438173


In [311]:
display_topics(NMF_tfidf, tfidf_feature_names, 10)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,thank,4.5,easy,1.0,condition,4.0,gift,4.1,series,4.2,wonderful,4.1,nice,3.8,helpful,3.8,enjoy,3.6,perfect,2.2
1,condition,0.2,lot,0.8,arrive,0.5,christmas,0.2,gift,0.2,author,0.1,wonderful,0.4,informative,0.2,helpful,0.3,old,2.0
2,service,0.2,work,0.7,perfect,0.4,perfect,0.1,enjoy,0.2,condition,0.1,series,0.2,series,0.2,son,0.2,year,1.9
3,product,0.2,fun,0.7,describe,0.2,friend,0.1,character,0.1,gift,0.0,picture,0.1,information,0.1,old,0.2,daughter,0.7
4,item,0.1,information,0.7,price,0.1,purchase,0.1,wait,0.1,message,0.0,quality,0.1,easy,0.1,year,0.2,son,0.3
5,describe,0.1,author,0.7,receive,0.1,daughter,0.1,favorite,0.1,man,0.0,gift,0.1,guide,0.1,kid,0.1,grandson,0.2
6,price,0.1,recipe,0.6,come,0.1,husband,0.1,condition,0.1,miss,0.0,calendar,0.0,gift,0.1,reading,0.1,kid,0.2
7,perfect,0.1,new,0.6,pleased,0.1,birthday,0.1,end,0.1,world,0.0,condition,0.0,study,0.1,nice,0.1,helpful,0.2
8,new,0.1,life,0.6,brand,0.1,receive,0.1,son,0.1,writer,0.0,addition,0.0,useful,0.1,cute,0.1,favorite,0.1
9,delivery,0.1,way,0.5,item,0.0,recipient,0.1,entire,0.1,collection,0.0,delivery,0.0,understand,0.1,wonderful,0.1,granddaughter,0.1


In [312]:
data = display_topics(NMF_tfidf, tfidf_feature_names, 10)
fig = make_subplots(rows=5, cols=2, subplot_titles= ["Topic 1", "Topic 6", "Topic 2","Topic 7", "Topic 3","Topic 8","Topic 4","Topic 9", "Topic 5", "Topic 10"])
colors = ["Blue", "Pink", "SeaGreen", "Orange", "Grey", "Purple", "Red", "LightBlue", "LightGreen", "DarkRed"]
for col in range(0, int(len(data.columns)),2):
    if col == 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,1].astype("float64"),#.sort_values(ascending=True)
        y=data.iloc[:,0], #.sort_values( by= data.columns[col+1], ascending = True)
        marker_color=colors[0], orientation = "h"), col= 1, row = 1)
    if col < 10 and col > 0:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"), col= 1, row = int(col/2)+1)
    if col >= 10:
        fig.add_trace(go.Bar(
        x=data.iloc[:,col+1].astype("float64"),
        y=data.iloc[:,col],
        marker_color=colors[int(col/2)], orientation = "h"),col= 2, row = int(col/2)-4)

fig.update_layout( height=1400, width=1200, title_text="NMF TF-IDF - Topic Keywords ranked by Weight", showlegend=False, xaxis_title_text='Keyword',yaxis_title_text='Weight'
    )
fig.update_yaxes(title='Keyword', type = "category" , autorange = "reversed")
fig.update_xaxes( title = 'Weight', range = (0,5) )
fig.show()

# Finding Best k Number of Clusters

Hyperparameters were drawn from literature. 
-> directly from gensim and Applying LDA Topic Modeling in Communication Research: Toward a Valid and Reliable Methodology by Maier

## LSA - TF

In [64]:
n_components = [2,4,6,8,10, 15, 20, 25, 30, 35, 40, 45, 50]
eval_lsa_tf = find_best_k(n_components,"LSA", tf_fit, tf, tf_feature_names, 20, df['cut_and_clean'], dictionary)


Coherence Score CV:  0.425267019061171

Coherence Score UMass:  -2.736941575998885

Coherence Score NPMI:  -0.0608608900527072

Coherence Score UCI:  -2.212516710746052

Silhouette Score:  -0.07176991290309291

Calinski-Harabasz Score:  10.243917936248401

Davies-Bouldin Score:  7.9914471816405745

Coherence Score CV:  0.5234024051442624

Coherence Score UMass:  -2.870075712617679

Coherence Score NPMI:  -0.011345358601791217

Coherence Score UCI:  -1.4320669329907358

Silhouette Score:  -0.10178055527585675

Calinski-Harabasz Score:  10.199602532621377

Davies-Bouldin Score:  8.051679888179775

Coherence Score CV:  0.5462720479836111

Coherence Score UMass:  -3.0487001477097917

Coherence Score NPMI:  -0.017130510824304435

Coherence Score UCI:  -1.7052997370986132

Silhouette Score:  -0.11301619976852742

Calinski-Harabasz Score:  12.681905988276057

Davies-Bouldin Score:  8.728108043188085

Coherence Score CV:  0.5233496008335523

Coherence Score UMass:  -3.0530641092224755

Cohere

In [49]:
eval_lsa_tf = pd.read_csv('../Results/bestk_lsa_tf_arpack.csv').set_index("Unnamed: 0")

In [247]:
plot_results(eval_lsa_tf, "Results LSA with TF")

## LSA TF-IDF

In [67]:
n_components = [2,4,6,8,10, 15, 20, 25, 30, 35, 40, 45, 50]
eval_lsa_tfidf = find_best_k(n_components,"LSA", tfidf_fit, tfidf, tfidf_feature_names, 20, df['cut_and_clean'], dictionary)


Coherence Score CV:  0.3711541130792214

Coherence Score UMass:  -3.217180714010678

Coherence Score NPMI:  0.014239499382799118

Coherence Score UCI:  0.0737120949837644

Silhouette Score:  -0.07984826774516944

Calinski-Harabasz Score:  113.10943351704103

Davies-Bouldin Score:  6.435643815754174

Coherence Score CV:  0.36350703925505684

Coherence Score UMass:  -3.587089703041029

Coherence Score NPMI:  0.04637268249392017

Coherence Score UCI:  0.14743219542335478

Silhouette Score:  -0.07579195989199396

Calinski-Harabasz Score:  131.21336920146874

Davies-Bouldin Score:  5.094351123583723

Coherence Score CV:  0.38150268349868516

Coherence Score UMass:  -3.5475773075305153

Coherence Score NPMI:  0.02193178608040164

Coherence Score UCI:  -0.2624624865086285

Silhouette Score:  -0.06909739599342625

Calinski-Harabasz Score:  131.116268779284

Davies-Bouldin Score:  4.687334851913598

Coherence Score CV:  0.4078163061147305

Coherence Score UMass:  -3.5894288184393917

Coherence

In [269]:
eval_lsa_tfidf = pd.read_csv('../Results/bestk_lsa_tfidf_arpack.csv').set_index("Unnamed: 0")

In [270]:

plot_results(eval_lsa_tfidf, "Results LSA with TF-IDF")

## LDA TF

In [70]:
n_components = [2,4,6,8,10, 15, 20, 25, 30, 35, 40, 45, 50]
eval_lda_tf = find_best_k(n_components,"LDA", tf_fit, tf, tf_feature_names, 20, df['cut_and_clean'], dictionary)


Coherence Score CV:  0.3994719813132219

Coherence Score UMass:  -2.3999356361398103

Coherence Score NPMI:  -0.005286439539325485

Coherence Score UCI:  -0.06143101849510031

Silhouette Score:  0.028091287172420955

Calinski-Harabasz Score:  46.46415418103314

Davies-Bouldin Score:  14.591858720859726

Coherence Score CV:  0.4862293976680549

Coherence Score UMass:  -2.6467849226660913

Coherence Score NPMI:  0.016581040746009773

Coherence Score UCI:  0.08778906967453522

Silhouette Score:  -0.03601273409164889

Calinski-Harabasz Score:  43.51197129654778

Davies-Bouldin Score:  12.300181551717841

Coherence Score CV:  0.494427792208917

Coherence Score UMass:  -2.684130544244424

Coherence Score NPMI:  0.0325136376108365

Coherence Score UCI:  0.19443848908589736

Silhouette Score:  -0.1078797056484464

Calinski-Harabasz Score:  33.183104577973936

Davies-Bouldin Score:  12.805879996982634

Coherence Score CV:  0.4924863528277992

Coherence Score UMass:  -2.8106624166120167

Cohere

In [271]:
eval_lda_tf = pd.read_csv('../Results/bestk_lda_tf.csv').set_index("Unnamed: 0")

In [272]:


plot_results(eval_lda_tf, "Results LDA with TF")

## LDA TF-IDF

In [78]:
n_components = [2,4,6,8,10, 15, 20, 25, 30, 35, 40, 45, 50]
eval_lda_tfidf = find_best_k(n_components, "LDA", tfidf_fit, tfidf, tfidf_feature_names, 20, df['cut_and_clean'], dictionary)


Coherence Score CV:  0.3832235567977873

Coherence Score UMass:  -3.1974645814717624

Coherence Score NPMI:  -0.013685621229843799

Coherence Score UCI:  -0.15146234301582695

Silhouette Score:  0.004170832360680667

Calinski-Harabasz Score:  32.529340783832

Davies-Bouldin Score:  24.43579120879118

Coherence Score CV:  0.4321151305411258

Coherence Score UMass:  -3.108112201769347

Coherence Score NPMI:  -0.003764235651671978

Coherence Score UCI:  -0.0889849046977047

Silhouette Score:  -0.011693917996888112

Calinski-Harabasz Score:  23.248401326930576

Davies-Bouldin Score:  21.896992504206615

Coherence Score CV:  0.40678375437014763

Coherence Score UMass:  -3.1518215982641173

Coherence Score NPMI:  -0.007813616702243472

Coherence Score UCI:  -0.15609866284764792

Silhouette Score:  -0.0264555533414533

Calinski-Harabasz Score:  20.034972629665663

Davies-Bouldin Score:  20.12839954352513

Coherence Score CV:  0.4151012927851969

Coherence Score UMass:  -3.1136162998685877

C

In [273]:
eval_lda_tfidf = pd.read_csv('../Results/bestk_lda_tfidf.csv').set_index("Unnamed: 0")

In [274]:


plot_results(eval_lda_tfidf, "Results LDA with TF-IDF")

## NMF TF

In [81]:
n_components = [2,4,6,8, 10, 15, 20, 25, 30, 35, 40, 45, 50]
eval_nmf_tf = find_best_k(n_components,"NMF", tf_fit, tf, tf_feature_names, 20, df['cut_and_clean'], dictionary)

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.




Coherence Score CV:  0.4637562995472074

Coherence Score UMass:  -2.8479450461797655

Coherence Score NPMI:  -0.031479462049109355

Coherence Score UCI:  -1.377508328057106

Silhouette Score:  0.005884102515331075

Calinski-Harabasz Score:  19.026612094381182

Davies-Bouldin Score:  7.587460249168673


C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.




Coherence Score CV:  0.4936294908072495

Coherence Score UMass:  -2.9001588477366393

Coherence Score NPMI:  -0.010365330409630046

Coherence Score UCI:  -0.9421838952343602

Silhouette Score:  -0.011366095881916057

Calinski-Harabasz Score:  18.535243287478355

Davies-Bouldin Score:  9.38270801984617


C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.



...


Coherence Score CV:  0.5221692081769371

Coherence Score UMass:  -3.2529767791371844

Coherence Score NPMI:  -0.0005763630559935285

Coherence Score UCI:  -1.0268371873076503

Silhouette Score:  -0.1858239723210872

Calinski-Harabasz Score:  31.663252275827936

Davies-Bouldin Score:  5.519101049422917


C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.




Coherence Score CV:  0.53407271222607

Coherence Score UMass:  -3.241556849160343

Coherence Score NPMI:  0.00961750334178314

Coherence Score UCI:  -0.8387225325105934

Silhouette Score:  -0.20969568843884576

Calinski-Harabasz Score:  29.796276283417047

Davies-Bouldin Score:  5.350817371068179


C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.




Coherence Score CV:  0.5205811833569137

Coherence Score UMass:  -3.242545435558928

Coherence Score NPMI:  0.0026940062869658487

Coherence Score UCI:  -0.9369054410865328

Silhouette Score:  -0.21448995202833102

Calinski-Harabasz Score:  29.149172991252488

Davies-Bouldin Score:  5.226247630641113
FINISHED! Evaluation of 20.000 Documents for topic modelling are evaluated!


In [275]:
eval_nmf_tf = pd.read_csv('../Results/bestk_nmf_tf_nndsvd_batch.csv').set_index("Unnamed: 0")

In [276]:

plot_results(eval_nmf_tf, "Results NMF with TF")

In [84]:
n_components = [2,4,6,8, 10, 15, 20, 25, 30, 35, 40, 45, 50]
eval_nmf_tfidf = find_best_k(n_components,"NMF", tfidf_fit, tfidf, tfidf_feature_names, 20, df['cut_and_clean'], dictionary)

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.




Coherence Score CV:  0.36115851841230706

Coherence Score UMass:  -3.1337442300470295

Coherence Score NPMI:  0.027711820068063872

Coherence Score UCI:  0.16553547474713698

Silhouette Score:  -0.08996231169382776

Calinski-Harabasz Score:  115.33187804323329

Davies-Bouldin Score:  6.00448652824122


C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.




Coherence Score CV:  0.3543589453431503

Coherence Score UMass:  -4.2165571075388755

Coherence Score NPMI:  0.03266201743339853

Coherence Score UCI:  -0.2791297732044661

Silhouette Score:  -0.09136399806430213

Calinski-Harabasz Score:  129.7410448952489

Davies-Bouldin Score:  5.149862882763653


C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.



...


Coherence Score CV:  0.39894915927756874

Coherence Score UMass:  -3.6341830259550436

Coherence Score NPMI:  0.0067030243746804665

Coherence Score UCI:  -0.7519891413106403

Silhouette Score:  -0.07757391370687526

Calinski-Harabasz Score:  77.9230406649918

Davies-Bouldin Score:  4.61151592424093


C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.




Coherence Score CV:  0.3983235403491663

Coherence Score UMass:  -3.5809463794327856

Coherence Score NPMI:  0.007550137457891649

Coherence Score UCI:  -0.6805668867334623

Silhouette Score:  -0.07519938915793282

Calinski-Harabasz Score:  74.73485150345672

Davies-Bouldin Score:  4.575389889888573


C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:1466: UserWarning:

The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.




Coherence Score CV:  0.3911030677781278

Coherence Score UMass:  -3.726610188843157

Coherence Score NPMI:  -0.006601948833400528

Coherence Score UCI:  -1.061254502328038

Silhouette Score:  -0.0710244965550436

Calinski-Harabasz Score:  71.50429684048358

Davies-Bouldin Score:  4.449589080416305
FINISHED! Evaluation of 20.000 Documents for topic modelling are evaluated!


In [278]:
eval_nmf_tfidf = pd.read_csv('../Results/bestk_nmf_tfidf_nndsvd_batch.csv',index_col="Unnamed: 0")
plot_results(eval_nmf_tfidf, "Results NMF with TF-IDF")